In [1]:
import os, sys
root = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
sys.path.append(root)

import Auto3D
from Auto3D.auto3D import options, main

In [2]:
#Always ensure that you have the latest version
print(Auto3D.__version__)

0.1.9


# Generate low-energy 3D structures with Auto3D

In [3]:
if __name__ == "__main__":
    path = os.path.join(root, "example/files/smiles.smi")
    args = options(path, k=1, use_gpu=False)   #args specify the parameters for Auto3D 
    out = main(args)            #main acceps the parameters and run Auto3D
    print(out)

Checking input file...
	There are 4 SMILES in the input file /Users/liu5/Documents/Auto3D_pkg/example/files/smiles.smi. 
	All SMILES and IDs are valid.
Suggestions for choosing isomer_engine and optimizing_engine: 
	Isomer engine options: RDKit and Omega.
	Optimizing engine options: ANI2x, ANI_2xt and AIMNET.
There are 4 SMILES, available memory is 16.0 GB.
The task will be divided into 1 jobs.
Job1, number of inputs: 4


Isomer generation for job1
Enumerating conformers/rotamers, removing duplicates...


Optimizing on job1
Preparing for parallel optimizing... (Max optimization steps: 5000)
Total 3D conformers: 14


 10%|▉         | 498/5000 [00:34<02:56, 25.49it/s]

Total 3D structures: 14  Converged: 9   Active: 5


 14%|█▍        | 701/5000 [00:38<03:58, 18.05it/s]


All structures converged at step 702:   Total 3D structures: 14  Converged: 14   Active: 0
Beggin to slelect structures that satisfy the requirements...
Energy unit: Hartree if implicit.
Program running time: 0 minutes
/Users/liu5/Documents/Auto3D_pkg/example/files/20220627-110531_smiles/smiles_out.sdf
